In [1]:
import wfdb
import ast
import numpy as np
import pandas as pd

In [2]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

In [3]:
basedir = '/home/ubuntu/classification/dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3/'
sampling_rate=100

In [4]:
# load and convert annotation data
Y = pd.read_csv(basedir+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [5]:
# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(basedir+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

In [6]:
#Apply diagnostic superclass function to find the class labels
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    # print(set(tmp))
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)
Y = Y.explode('diagnostic_superclass')
Y = Y.dropna(subset=['diagnostic_superclass'])
Y.to_csv('ptbxl_info.csv', index_label = 'ecg_id')

In [7]:
# Load raw signal data and save the data in numpy format
X = load_raw_data(Y, sampling_rate, basedir)
with open ('lead_readings_1.npy','wb') as f:
    np.save(f,X)